In [3]:
# https://stackoverflow.com/questions/18621513/python-insert-numpy-array-into-sqlite3-database
import sqlite3 as sql
import numpy as np
from sqlite3 import Error
import io
import os

database = "test.db"

def adapt_array(arr):
    """
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)


# Converts np.array to TEXT when inserting
sqlite3.register_adapter(np.ndarray, adapt_array)

# Converts TEXT to np.array when selecting
sqlite3.register_converter("ARRAY", convert_array)

x = np.arange(16).reshape(4,4)
y = np.arange(4)

try:
    with sql.connect(database, detect_types=sqlite3.PARSE_DECLTYPES) as conn:
        cur = conn.cursor()
        cur.execute("CREATE TABLE IF NOT EXISTS Memory (State ARRAY, Probabilites ARRAY, Score INTEGER)") # Creates new tables with specific column names
        cur.execute("INSERT INTO Memory VALUES(?,?,?);", (x,y,2)) # Inserts data into each of the tables
        cur.execute("SELECT * from Memory")
        data = cur.fetchone()
        print(data)
finally:
    conn.commit()
    conn.close()

(array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]]), array([0, 1, 2, 3]), 2)


In [82]:
os.path.isfile(database)

True

TypeError: 'NoneType' object is not subscriptable

In [16]:
import sqlite3
import numpy as np
from sqlite3 import Error
import io

# Modified from http://www.sqlitetutorial.net/sqlite-python/create-tables/

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def main():
    database = r"~\sqlite\db\pythonsqlite.db"
 
    sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS projects (
                                        id integer PRIMARY KEY,
                                        name text NOT NULL,
                                        begin_date text,
                                        end_date text
                                    ); """
 
    sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS tasks (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    priority integer,
                                    status_id integer NOT NULL,
                                    project_id integer NOT NULL,
                                    begin_date text NOT NULL,
                                    end_date text NOT NULL,
                                    FOREIGN KEY (project_id) REFERENCES projects (id)
                                );"""
 
    # create a database connection
    conn = create_connection(database)
 
    # create tables
    if conn is not None:
        # create projects table
        create_table(conn, sql_create_projects_table)
 
        # create tasks table
        create_table(conn, sql_create_tasks_table)
        conn.commit()
    else:
        print("Error! cannot create the database connection.")
        
main()

~\sqlite\db\pythonsqlite.db


In [6]:
data


(array([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]]),)

In [47]:
import sqlite3 as sql
import numpy as np
from sqlite3 import Error
import io

database = r"~\sqlite\db\test.db"

try:
    with sql.connect(database) as conn:
        cur = conn.cursor()
        cur.execute("DROP TABLE IF EXISTS Memory") # Drops table names if they exist
        cur.execute("CREATE TABLE Memory (State ARRAY, Probabilities ARRAY, Score INTEGER)") # Creates new tables with specific column names
        cur.executemany("INSERT INTO Memory VALUES(?,?,?);", [(1,2,3)]) # Inserts data into each of the tables

finally:
    conn.commit()
    conn.close()
        